In [37]:
import pandas as pd
import re
import time
import pickle

training_data = pd.read_excel("NPD_Data.xlsx", header=0).values

In [38]:
# # Divides mapped from unmapped
# testing_data = amazon[amazon[:,3] != "Unmapped"]
# amazon_unmapped = amazon[amazon[:,3] == "Unmapped"]

In [40]:
# Divides description of each product into individual words
def splitter(df):
    split = [0]*df.shape[0]
    # Useless strings to be removed
    purge_list = ["","a","and","with","for","in","the","of","to","x","disney","-","&"]
    
    # Divides description into individual strings
    for i in range(0,df.shape[0]):
        split[i] = [x.lower() for x in (re.split('[ _,+\'/#?:;()|]',str(df[i][0])))]
        
        for j in purge_list:
            while(j in split[i]): 
                split[i].remove(j)
    return split

In [41]:
training_corpus = splitter(training_data)

In [42]:
# Not super useful, just counts occurences of each term in the entire sheet
def count_occurrences(df):
    dict = {}
    for i in range(0,len(df)):
        for j in range(0,len(df[i])):
            if df[i][j] not in dict:
                dict[df[i][j]] = {}
            if i not in dict[df[i][j]]:
                dict[df[i][j]][i] = 0
            dict[df[i][j]][i] += 1
    return dict

In [43]:
training_count = count_occurrences(training_corpus)

In [44]:
# Finds occurences of each property, e.g. "Frozen": 2000
def count_properties(dic):
    count = {}
    for i in dic:
        count[i] = 0
        for j in dic[i]:
            count[i] += dic[i][j]
    return count

In [45]:
CP = count_properties(training_count)
# Sorts dictionary of properties into order of occurrence
CP = {k: v for k, v in sorted(CP.items(), key=lambda item: item[1], reverse=True)}

In [46]:
# Lists to be used for training
X = training_data[:,1]
y = training_corpus

In [10]:
# This will take FOREVER, so just let it sit and run for a good 1h 20mins
count = 0
trainingDict = {}
# Go through all the properties and the list of products and add each indivdual word of item description to the property it associates with and tally the occurences of this word
# Word will then build an "identity of words that are commonly used with it.
load = len(CP)
prev = 0
start_time = time.time()
for listKeyword in CP:
    if round((count/load)*10) != prev:
            print(round((count/load)*10)*10, "% complete...")
            t = time.time() - start_time
            print("Elapsed time:", round((t)-(t%1))/60, "minutes\n")
            prev = round((count/load)*10)
    for j in range(len(y)):
        for keyword in y[j]:
            if listKeyword == keyword:
                if X[j] not in trainingDict:
                    trainingDict[X[j]] = {}
                if listKeyword not in trainingDict[X[j]]:
                    trainingDict[X[j]][listKeyword] = 1
                else:
                    trainingDict[X[j]][listKeyword] += 1
    count += 1

10 % complete...
Elapsed time: 3.6166666666666667 minutes

20 % complete...
Elapsed time: 10.8 minutes

30 % complete...
Elapsed time: 18.183333333333334 minutes

40 % complete...
Elapsed time: 25.816666666666666 minutes

50 % complete...
Elapsed time: 35.28333333333333 minutes

60 % complete...
Elapsed time: 47.0 minutes

70 % complete...
Elapsed time: 56.36666666666667 minutes

80 % complete...
Elapsed time: 66.11666666666666 minutes

90 % complete...
Elapsed time: 74.48333333333333 minutes

100 % complete...
Elapsed time: 82.81666666666666 minutes



In [48]:
for i in list(trainingDict):
    for j in list(trainingDict[i]):
        if trainingDict[i][j] == 1:
            del trainingDict[i][j]

del trainingDict["All Other"]
del trainingDict["Other License"]
# Number of unique properties used in training set
print(len(trainingDict))
pickle.dump(trainingDict, open("TrainingData.p","wb"))

# Will print FAT list of properties and list of words used with it. Just scroll past it.
# for i in trainingDict:
#     print(i)
#     print(trainingDict[i])
#     print("\n")

2704
